In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
import gensim
import skipthoughts
import re           
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from sklearn.cluster import MeanShift

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = ['.', ',', '!', '?', ';', ':']

def remove_stopwords(text):
    filtered_text = [w for w in text.split() if not w in stop_words]
    return filtered_text

def remove_punctuation(text):
    filtered_text = ''.join([w for w in text if w not in punctuation])
    return filtered_text

def map_contractions(text):
    filtered_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return filtered_text

In [5]:
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return lemmatized_text

def word_stemmer(text):
    stemmed_text = PorterStemmer(text)
    return stemmed_text

In [6]:
def clean_text(text):
    cleaned_text = text.lower()
    cleaned_text = map_contractions(cleaned_text)
    #cleaned_text = word_stemmer(cleaned_text)
    #cleaned_text = word_lemmatizer(cleaned_text)
    return cleaned_text

In [7]:
doc = "She reached her goal, exhausted. Even more chilling to her was that the euphoria that she thought she'd feel upon reaching it wasn't there. Something wasn't right. Was this the only feeling she'd have for over five years of hard work?"
cleaned_text = clean_text(doc)
cleaned_sentences = sent_tokenize(cleaned_text)

In [8]:
cleaned_sentences

['she reached her goal, exhausted.',
 'even more chilling to her wa that the euphoria that she thought she would feel upon reaching it wa not there.',
 'something wa not right.',
 'wa this the only feeling she would have for over five year of hard work?']

In [9]:
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

In [10]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [11]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

2020-05-28 17:41:26,508 : WARNING : consider setting layer size to a multiple of 4 for greater performance
C:\Users\Felix\Anaconda3\envs\betaLab\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [12]:
model.build_vocab(train_corpus)

2020-05-28 17:41:26,584 : INFO : collecting all words and their counts
2020-05-28 17:41:26,590 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-28 17:41:26,663 : INFO : collected 6981 word types and 300 unique tags from a corpus of 300 examples and 58152 words
2020-05-28 17:41:26,669 : INFO : Loading a fresh vocabulary
2020-05-28 17:41:26,719 : INFO : effective_min_count=2 retains 3955 unique words (56% of original 6981, drops 3026)
2020-05-28 17:41:26,726 : INFO : effective_min_count=2 leaves 55126 word corpus (94% of original 58152, drops 3026)
2020-05-28 17:41:26,778 : INFO : deleting the raw counts dictionary of 6981 items
2020-05-28 17:41:26,781 : INFO : sample=0.001 downsamples 46 most-common words
2020-05-28 17:41:26,786 : INFO : downsampling leaves estimated 42390 word corpus (76.9% of prior 55126)
2020-05-28 17:41:26,819 : INFO : estimated required memory for 3955 words and 50 dimensions: 3619500 bytes
2020-05-28 17:41:26,823 : INFO : res

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-05-28 17:41:28,963 : INFO : training model with 3 workers on 3955 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-05-28 17:42:03,271 : INFO : EPOCH 1 - PROGRESS: at 16.67% examples, 205 words/s, in_qsize 5, out_qsize 0
2020-05-28 17:42:06,026 : INFO : EPOCH 1 - PROGRESS: at 32.00% examples, 389 words/s, in_qsize 4, out_qsize 0
2020-05-28 17:42:42,676 : INFO : EPOCH 1 - PROGRESS: at 66.67% examples, 387 words/s, in_qsize 2, out_qsize 1
2020-05-28 17:42:42,684 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-28 17:42:45,164 : INFO : EPOCH 1 - PROGRESS: at 83.00% examples, 464 words/s, in_qsize 1, out_qsize 1
2020-05-28 17:42:45,176 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-28 17:42:45,283 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-28 17:42:45,284 : INFO : EPOCH - 1 : training on 58152 raw words (42426 effective words) took 76.3s, 556 effective words/s
2020-05-

2020-05-28 17:52:47,063 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-28 17:52:52,696 : INFO : EPOCH 10 - PROGRESS: at 100.00% examples, 701 words/s, in_qsize 0, out_qsize 1
2020-05-28 17:52:52,700 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-28 17:52:52,700 : INFO : EPOCH - 10 : training on 58152 raw words (42417 effective words) took 60.5s, 701 effective words/s
2020-05-28 17:53:25,296 : INFO : EPOCH 11 - PROGRESS: at 18.67% examples, 219 words/s, in_qsize 5, out_qsize 0
2020-05-28 17:53:27,155 : INFO : EPOCH 11 - PROGRESS: at 34.00% examples, 421 words/s, in_qsize 4, out_qsize 0
2020-05-28 17:53:31,239 : INFO : EPOCH 11 - PROGRESS: at 50.67% examples, 561 words/s, in_qsize 3, out_qsize 0
2020-05-28 17:53:59,003 : INFO : EPOCH 11 - PROGRESS: at 66.67% examples, 431 words/s, in_qsize 2, out_qsize 1
2020-05-28 17:53:59,003 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-28 17:54:00,842 : INFO : EPOCH 

In [ ]:
vectors = []

for sentence in cleaned_sentences:
    words = sentence.split()
    vectors.append(model.infer_vector(words))


In [ ]:
vectors

In [ ]:
clusters = MeanShift().fit(vectors)
labels = clusters.labels_
centers = clusters.labels = clusters.labels_

In [ ]:
keep_labels = []

for i in range(len(centers)):
    indeces = np.where(labels == i)
    most_centered = indeces[0][0]
    if len(indeces[0]) == 1:
        keep_labels.append(most_centered)
    else:
        for j in range(len(indeces)):
            shortest_distance = inf
            distance = euclidean(centers[labels[i]], vectors[indeces[0][j]])
            if distance < shortest_distance:
                shortest_distance = distance
                most_centered = indeces[0][j]
        keep_labels.append(most_centered)
        
keep_labels

In [ ]:
keep_sentences = []

for i in range(len(cleaned_sentences)):
    if i in keep_labels:
        keep_sentences.append(cleaned_sentences[i])

summarized_text = ' '.join(sentence for sentence in keep_sentences)   

summarized_text